In [2]:
# ! pip install git+https://github.com/facebookresearch/ImageBind
! pip install soundfile
! pip install librosa
! pip install "imagebind @ git+https://github.com/facebookresearch/ImageBind@c6a47d6dc2b53eced51d398c181d57049ca59286"

  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)


In [2]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
# create triplet dataset
from datasets import load_dataset
import soundfile as sf
import requests
import pandas as pd
# https://huggingface.co/datasets/agkphysics/AudioSet
# dataset = load_dataset("agkphysics/AudioSet", "bal", split="test")
dataset = load_dataset("arrow", split="test", data_files={"test":"C:/Users/Christina/.cache/huggingface/datasets/agkphysics___audio_set/bal/audio_set-test-000**-of-00018.arrow"})#, split="test[0:10]")
# https://huggingface.co/docs/datasets/v2.16.1/en/package_reference/main_classes#datasets.Dataset.filter
dataset_it = dataset.to_iterable_dataset()

data_dir = "C:\\Users\\Christina\\Data\\imagebind\\text-audio-image\\"
animals = ["Bird", "Cat", "Dog", "Horse"]

infos = []
idx = 0
for animal in animals:
    for example in dataset_it.filter(lambda x: animal in x["human_labels"]).take(25):
        # save audiofile
        sf.write(data_dir + "audio\\%03d.wav"%idx, example["audio"]["array"], example["audio"]["sampling_rate"], format="wav")

        # save image
        # https://stackoverflow.com/questions/2068344/how-do-i-get-a-youtube-video-thumbnail-from-the-youtube-api
        url = 'https://img.youtube.com/vi/%s/hqdefault.jpg'%example["video_id"]
        data = requests.get(url).content 
        with open(data_dir + "image\\%03d.jpg"%idx,'wb') as f:
            f.write(data) 

        # save info
        infos.append({"id": idx, "video_id":example["video_id"], "labels":example["human_labels"], "animal": animal})

        idx += 1

pd.DataFrame(infos).to_csv(data_dir + "info.csv", index=False)

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [36]:
# # https://huggingface.co/docs/datasets/loading#slice-splits
# from datasets import load_dataset

# # dataset = load_dataset("agkphysics/AudioSet", "bal", split="test") #, streaming=True
# dataset = load_dataset("arrow", split="test", data_files={"test":"C:/Users/Christina/.cache/huggingface/datasets/agkphysics___audio_set/bal/audio_set-test-000**-of-00018.arrow"})#, split="test[0:10]")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


# Image-Text

In [4]:
from amumo import model as am_model
from amumo import data as am_data
from amumo import utils as am_utils
from amumo import widgets as am_widgets

# load dataset
dataset = am_data.DiffusionDB_Dataset(path="2m_first_1k", batch_size=100) # data helper for the diffusionDB dataset; for the interactive prototype, we only use a random subset of 100 samples
all_images, all_prompts = dataset.get_data()
cache_name = 'diffusiondb_random_100' # path used to cache the results

KeyboardInterrupt: 

In [5]:
am_widgets.CLIPExplorerWidget(cache_name, all_data={"image": all_images, "text": all_prompts}, models=[am_model.ImageBind_Model(), "CLIP"]) # {"image": all_images, "depth": all_depths}

100%|███████████████████████████████████████| 244M/244M [01:27<00:00, 2.92MiB/s]


found cached embeddings for diffusiondb_random_100_ImageBind_huge image
found cached embeddings for diffusiondb_random_100_ImageBind_huge text


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

found cached embeddings for diffusiondb_random_100_CLIP_RN50 image
found cached embeddings for diffusiondb_random_100_CLIP_RN50 text
found cached embeddings for diffusiondb_random_100_ImageBind_huge image
found cached embeddings for diffusiondb_random_100_ImageBind_huge text


In [6]:
am_widgets.CLIPComparerWidget(cache_name, all_images=all_images, all_prompts=all_prompts, models=[am_model.ImageBind_Model(), "CLIP"]) # {"image": all_images, "depth": all_depths}

True [False, False] {'ImageBind': <amumo.model.ImageBind_Model object at 0x000001B878F032E0>, 'CLIP': <amumo.model.CLIPModel object at 0x000001B767795E20>}


c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:940: FutureWarning:

The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:940: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



found cached embeddings for diffusiondb_random_100_ImageBind_huge
found cached embeddings for diffusiondb_random_100_CLIP_RN50


CLIPComparerWidget(children=(HoverWidget(children=(VBox(children=(HTML(value='', layout=Layout(width='300px'))…

# Image-Text-Audio

In [5]:
from amumo import model as am_model
from amumo import data as am_data
from amumo import utils as am_utils
from amumo import widgets as am_widgets
# ! pip install torch
# ! pip install torchaudio
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

In [1]:
from amumo import model as am_model
from amumo import data as am_data
from amumo import utils as am_utils
from amumo import widgets as am_widgets
# ! pip install torch
# ! pip install torchaudio
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"


from glob import glob
from PIL import Image
import numpy as np
import io
import torchaudio
import pandas as pd


class AudioType(am_data.DataTypeInterface):
    name = "Audio"

    def __init__(self, data, sample_rate) -> None:
        super().__init__(data)
        self.sample_rate = sample_rate
    
    def getVisItem(self, idx):
        import soundfile as sf
        buffer = io.BytesIO()
        sf.write(buffer, self.data[idx][0], self.sample_rate, format="wav")
        audio_bytes = buffer.getvalue() # wav bytes
        return {"displayType": am_data.DisplayTypes.AUDIO, "value": audio_bytes}


class Triplet_Dataset(am_data.DatasetInterface):
    name='Triplet'

    def __init__(self, path, seed=31415, batch_size=100, sample_rate=16000):
        # create triplet dataset if it does not exist
        super().__init__(path, seed, batch_size)
        # path: path to the triplet dataset
        image_paths = glob(path + "image\\*.jpg", recursive = True)
        audio_paths = glob(path + "audio\\*.wav", recursive = True)

        self.sample_rate = sample_rate
        
        all_images = []
        for image_path in image_paths:
            with open(image_path, "rb") as fopen:
                image = Image.open(fopen).convert("RGB")
                all_images.append(image)

        all_audios = []
        for audio_path in audio_paths:
            waveform, sr = torchaudio.load(audio_path)
            if sample_rate != sr:
                waveform = torchaudio.functional.resample(
                    waveform, orig_freq=sr, new_freq=sample_rate
                )
            all_audios.append(waveform)
        
        self.all_infos = pd.read_csv(path + "info.csv", converters={"labels": lambda x: x.strip("[]").replace("'","").split(", ")})

        # TODO... load on demand with a custom loader
        self.all_images = np.array(all_images)
        self.all_prompts = np.array(self.all_infos["labels"].map(lambda x: ", ".join(x)))
        self.all_audios = np.array(all_audios)
    
    
    def get_data(self):
        
        if self.batch_size is None:
            images = self.MODE1_Type(self.all_images)
            texts = self.MODE2_Type(self.all_prompts)
            audios = AudioType(self.all_audios, self.sample_rate)
        
            return images, texts, audios

        # create a random batch
        batch_idcs = self._get_random_subsample(len(self.all_images))

        images = self.MODE1_Type(self.all_images[batch_idcs])
        texts = self.MODE2_Type(self.all_prompts[batch_idcs])
        audios = AudioType(self.all_audios[batch_idcs], self.sample_rate)
        
        return images, texts, audios
    
        
    def get_filtered_data(self, filter_list, method=any):
        # filter_list: a list of strings that are used for filtering
        # method: any -> any substring given in filter_list is present; all -> all substrings must be contained in the string
        if filter_list is None or len(filter_list) <= 0:
            return self.get_data()

        subset_ids = np.array([i for i in range(len(self.all_prompts)) if method(substring in self.all_prompts[i].lower() for substring in filter_list)])
        if len(subset_ids) <= 0:
            print("no filter matches found")
            return [], [], []
        
        # create a random batch
        batch_idcs = self._get_random_subsample(len(subset_ids))
        subset_ids = subset_ids[batch_idcs]
        
        images = self.MODE1_Type(self.all_images[subset_ids])
        texts = self.MODE2_Type(self.all_prompts[subset_ids])
        audios = AudioType(self.all_audios[subset_ids], self.sample_rate)
        return images, texts, audios


C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

In [9]:
from amumo.utils import get_textual_label_for_cluster, get_embeddings_per_modality, get_similarities, get_similarities_all, get_cluster_sorting, get_modality_distance, calculate_val_loss, get_closed_modality_gap, get_modality_gap_normed, l2_norm, get_gap_direction


import traitlets
from ipywidgets import widgets
import plotly.graph_objects as go
import numpy as np
import math

class SimilarityHeatmapClusteringWidget(widgets.VBox):
    
    embedding = traitlets.Dict().tag(sync=True)
    value = traitlets.Any(np.zeros((6,6))).tag(sync=True)
    cluster = traitlets.Any().tag(sync=True)
    modality_labels = traitlets.List([]).tag(sync=True)
    cluster_label_data = None

    hover_idx = traitlets.List([]).tag(sync=True)


    def __init__(self, zmin=None, zmax=None, cluster_label_data=None, hover_callback=None):
        super(SimilarityHeatmapClusteringWidget, self).__init__()

        self.cluster_label_data = cluster_label_data
        self.size = 6
        self.hover_callback = hover_callback

        self.cluster_similarity_matrix_widget = widgets.Checkbox(
            value=False,
            description='Cluster matrix by similarity',
            disabled=False,
            indent=False
        )
        self.cluster_similarity_matrix_by_widget1 = widgets.Dropdown(
            options=self.modality_labels,
            description='between',
            style = {"description_width": "initial"},
            layout=widgets.Layout(width="150px")
        )
        self.cluster_similarity_matrix_by_widget2 = widgets.Dropdown(
            options=self.modality_labels,
            description='and',
            style = {"description_width": "initial"},
            layout=widgets.Layout(width="150px")
        )
        
        self.fig_widget = go.FigureWidget(data=[go.Heatmap(z=self.value, zmin=zmin, zmax=zmax)])
        self.heatmap = self.fig_widget.data[0]
        self.heatmap.hoverinfo = "text"
        self.fig_widget.update_layout(width=500, height=420,
            margin=dict(l=10, r=10, t=10, b=10),
        )
        self.fig_widget.update_yaxes(autorange='reversed', fixedrange=False)
        self.fig_widget.update_xaxes(fixedrange=False)
        self.fig_widget.layout.shapes = self._get_matrix_gridlines()

        self.heatmap.on_hover(self._hover_fn)
        
        self.cluster_similarity_matrix_widget.observe(self.onUpdateEmbedding, names='value')
        self.cluster_similarity_matrix_by_widget1.observe(self.onUpdateClusterBy, names='value')
        self.cluster_similarity_matrix_by_widget2.observe(self.onUpdateClusterBy, names='value')

        settings = widgets.HBox([self.cluster_similarity_matrix_widget, 
                                 self.cluster_similarity_matrix_by_widget1, 
                                 self.cluster_similarity_matrix_by_widget2], 
                                 layout=widgets.Layout(width="430px"))
        self.children = [settings, widgets.HBox([self.fig_widget])]



    def _hover_fn(self, trace, points, state):
        x_idx = points.xs[0]
        y_idx = points.ys[0]
            
        # show hover lines in similarity heatmap
        self.hover_idx = [(x_idx, y_idx)]

        # extract original x and y index and modalities for x and y; then call the callback function
        if self.hover_callback is not None:
            x_modality_idx = math.floor(x_idx/self.size)
            y_modality_idx = math.floor(y_idx/self.size)

            # x_modality = 'modality2'
            # if x_idx < self.size:
            #     x_modality = 'modality1'

            # y_modality = 'modality2'
            # if y_idx < len(self.idcs):
            #     y_modality = 'modality1'

            x_idx = self.idcs[x_idx%len(self.idcs)]
            y_idx = self.idcs[y_idx%len(self.idcs)]
            # self.hover_callback(x_idx, y_idx, x_modality, y_modality)
            self.hover_callback(x_idx, y_idx, self.modality_labels[x_modality_idx], self.modality_labels[y_modality_idx])

    def _get_matrix_gridlines(self):
        no_modalities = len(self.modality_labels)  

        line_style = dict(color="black", width=1)
        gridlines = []
        for i in range(no_modalities-1):
            horizontal_line = go.layout.Shape(type='line', x0=len(self.value)*(1+i)/no_modalities-0.5, y0=0-0.5, x1=len(self.value)*(1+i)/no_modalities-0.5, y1=len(self.value)-0.5, line=line_style)
            gridlines.append(horizontal_line)
            vertical_line = go.layout.Shape(type='line', y0=len(self.value)*(1+i)/no_modalities-0.5, x0=0-0.5, y1=len(self.value)*(1+i)/no_modalities-0.5, x1=len(self.value)-0.5, line=line_style)
            gridlines.append(vertical_line)

        return gridlines

    @traitlets.observe("modality_labels")
    def onUpdateModalityLabel(self, change):
        no_modalities = len(self.modality_labels)
        self.fig_widget.update_layout(
            xaxis = dict(
                tickmode = 'array',
                tickvals = [(i*2+1)/2 * self.size for i in range(no_modalities)], #[1*len(self.value)/4, 3*len(self.value)/4]
                ticktext = self.modality_labels
            ),
            yaxis = dict(
                tickmode = 'array',
                tickvals = [(i*2+1)/2 * self.size for i in range(no_modalities)], #[1*len(self.value)/4, 3*len(self.value)/4]
                ticktext = self.modality_labels
            ),
        )

        self.cluster_similarity_matrix_by_widget1.options = self.modality_labels
        if self.cluster_similarity_matrix_by_widget1.value is None:
            self.cluster_similarity_matrix_by_widget1.value = self.modality_labels[0]
        self.cluster_similarity_matrix_by_widget2.options = self.modality_labels
        if self.cluster_similarity_matrix_by_widget2.value is None:
            self.cluster_similarity_matrix_by_widget2.value = self.modality_labels[0]
    

    def onUpdateClusterBy(self, change):
        if self.cluster_similarity_matrix_by_widget1.value is not None and self.cluster_similarity_matrix_by_widget2.value is not None and self.cluster_similarity_matrix_widget.value:
            self.onUpdateEmbedding(change)


    @traitlets.validate("value")
    def _validate_value(self, proposal):
        # print("TODO: validate value")
        return proposal.value

    @traitlets.observe("value")
    def onUpdateValue(self, change):
        self.fig_widget.data[0].z = self.value
        self.fig_widget.layout.shapes = self._get_matrix_gridlines()

    @traitlets.validate("embedding")
    def _validate_embedding(self, proposal):
        # print("TODO: validate embedding")
        # for backwards compatibility map array to dict
        if isinstance(proposal.value, tuple):
            embeddings = {}
            embeddings["Modality1"] = proposal.value[0]
            embeddings["Modality2"] = proposal.value[1]
            return embeddings
        return proposal.value

    @traitlets.observe("embedding")
    def onUpdateEmbedding(self, change):
        self.size=len(list(self.embedding.values())[0])
        self.modality_labels = list(self.embedding.keys())
        no_modalities = len(self.embedding.keys())
        similarity_all = get_similarities_all(self.embedding)

        cluster_labels = []
        cluster_sizes = []

        if self.cluster_similarity_matrix_widget.value:
            similarity_between = self.embedding[self.cluster_similarity_matrix_by_widget1.value]
            similarity_and = self.embedding[self.cluster_similarity_matrix_by_widget2.value]
            similarity_cross_modal = get_similarities(torch.from_numpy(similarity_between), torch.from_numpy(similarity_and))
            self.idcs, clusters, clusters_unsorted = get_cluster_sorting(similarity_cross_modal)
            for c in set(clusters):
                cluster_size = np.count_nonzero(clusters==c)
                cluster_label = ""
                if self.cluster_label_data is not None:
                    cluster_label = self.cluster_label_data.getMinSummary(np.where(clusters_unsorted==c)[0])
                cluster_labels.append(cluster_label)
                cluster_sizes.append(cluster_size)
        else:
            self.idcs = np.arange(self.size) # TODO: use reverse idcs to get original order for interaction with other widgets

        # with heatmap_widget.batch_update():
        matrix_sort_idcs = np.concatenate([self.idcs + i*self.size for i in range(no_modalities)], axis=0) # [self.idcs, self.idcs+self.size] # need to do double index because we combined images and texts
        
        self.value = similarity_all[matrix_sort_idcs, :][:, matrix_sort_idcs]
        self.cluster = (cluster_labels, cluster_sizes)

        
    # @traitlets.validate("cluster")
    # def _validate_cluster(self, proposal):
        # takes a list of cluster labels + sizes
    #     print("TODO: validate cluster")
    #     return proposal.value

    @traitlets.observe("cluster")
    def onUpdateCluster(self, change):
        cluster_modality1_idx = self.modality_labels.index(self.cluster_similarity_matrix_by_widget1.value)
        cluster_modality2_idx = self.modality_labels.index(self.cluster_similarity_matrix_by_widget2.value)
        cluster_shapes = self._get_matrix_gridlines()
        labels, sizes = self.cluster
        offset = 0-0.5 # -0.5 because heatmap rectangles are drawn around [-0.5, 0.5]
        for (cluster_label, cluster_size) in zip(labels, sizes):
            if cluster_size > 5:
                textposition = 'middle left' if offset < len(self.value)/2/2 else 'middle right'

                # see https://plotly.com/python/shapes/
                cluster_shapes += [go.layout.Shape(
                    type='rect', 
                    x0=cluster_modality1_idx*self.size+offset, 
                    y0=cluster_modality2_idx*self.size+offset, 
                    x1=cluster_modality1_idx*self.size+offset+cluster_size, 
                    y1=cluster_modality2_idx*self.size+offset+cluster_size, 
                    label=dict(text=cluster_label, textposition=textposition, font=dict(size=10, color="white"), padding=np.log(cluster_size)*10), 
                    line=dict(width=1, color='white')
                )]

            offset += cluster_size
            
        self.fig_widget.layout.shapes = cluster_shapes


    @traitlets.observe("hover_idx")
    def onUpdateHoverIdx(self, change):
        shapes = [sh for sh in self.fig_widget.layout.shapes if sh.name != 'hover_idx' and sh.name != 'hover_idx']

        for (x_idx, y_idx) in self.hover_idx:
            if x_idx >= 0 and x_idx < len(self.value):
                shapes.append(go.layout.Shape(name='hover_idx', type='line', x0=x_idx, y0=0-0.5, x1=x_idx, y1=len(self.value)-0.5, line=dict(color="grey", width=1)))
            if y_idx >= 0 and y_idx < len(self.value):
                shapes.append(go.layout.Shape(name='hover_idx', type='line', y0=y_idx, x0=0-0.5, y1=y_idx, x1=len(self.value)-0.5, line=dict(color="grey", width=1)))
        
        self.fig_widget.layout.shapes = shapes


class HoverWidget(widgets.VBox):
    
    valueX = traitlets.Any().tag(sync=True)
    valueY = traitlets.Any().tag(sync=True)


    def __init__(self, width=300):
        super(HoverWidget, self).__init__()

        self.width = width

        output_dummy_img = io.BytesIO()
        Image.new('RGB', (self.width,self.width)).save(output_dummy_img, format="JPEG")
        self.img_widgets = {'valueX': widgets.Image(value=output_dummy_img.getvalue(), width=0, height=0), 
                            'valueY': widgets.Image(value=output_dummy_img.getvalue(), width=0)} #, height=0)}
        self.txt_widgets = {'valueX': widgets.HTML(value='', layout=widgets.Layout(width="%ipx"%self.width)), 
                            'valueY': widgets.HTML(value='', layout=widgets.Layout(width="%ipx"%self.width)), }
        self.wav_widgets = {'valueX': widgets.Audio(autoplay=True, layout=widgets.Layout(width="0px", height="0px")),
                            'valueY': widgets.Audio(autoplay=True, layout=widgets.Layout(width="0px", height="0px"))}

        self.children = [widgets.VBox(list(self.wav_widgets.values())), widgets.VBox(list(self.txt_widgets.values())), widgets.VBox(list(self.img_widgets.values()))]

        self.layout = widgets.Layout(width="%ipx"%(self.width+10), height="inherit")


    @traitlets.validate("value1", "value2")
    def _validate_value(self, proposal):
        # print("TODO: validate value1")
        return proposal.value
    
    def set_text(self, name, value):
        cur_img_widget = self.img_widgets[name]
        cur_txt_widget = self.txt_widgets[name]
        cur_wav_widget = self.wav_widgets[name]

        cur_txt_widget.value = "<div style='word-wrap: break-word;'>{}</div>".format(value)
        cur_img_widget.width = 0
        # cur_img_widget.height = 0
        cur_wav_widget.value = b'RIFF$\xe2\x04\x00WAVEfmt'
        cur_wav_widget.layout = widgets.Layout(width="0px", height="0px")

    def set_img(self, name, value):
        cur_img_widget = self.img_widgets[name]
        cur_txt_widget = self.txt_widgets[name]
        cur_wav_widget = self.wav_widgets[name]

        cur_img_widget.value = value
        cur_img_widget.width = self.width
        # cur_img_widget.height = self.width
        cur_txt_widget.value = ""
        cur_wav_widget.value = b'RIFF$\xe2\x04\x00WAVEfmt'
        cur_wav_widget.layout = widgets.Layout(width="0px", height="0px")

    def set_wav(self, name, value):
        cur_img_widget = self.img_widgets[name]
        cur_txt_widget = self.txt_widgets[name]
        cur_wav_widget = self.wav_widgets[name]

        cur_img_widget.width = 0
        cur_txt_widget.value = ""
        cur_wav_widget.value = value
        cur_wav_widget.layout = widgets.Layout(width="%ipx"%self.width, height="20px")


    @traitlets.observe("valueX", "valueY")
    def onUpdateValue(self, change):
        if type(change.new) is io.BytesIO:
            self.set_img(change.name, change.new.getvalue())

        elif type(change.new) is dict:
            if "displayType" in change.new:
                if change.new["displayType"] == am_data.DisplayTypes.IMAGE:
                    self.set_img(change.name, change.new["value"])
                elif change.new["displayType"] == am_data.DisplayTypes.AUDIO:
                    self.set_wav(change.name, change.new["value"])
                else:
                    self.set_text(change.name, change.new["value"])

            elif "value" in change.new:
                self.set_text(change.name, change.new["value"])
        else:
            self.set_text(change.name, change.new)
        



from sklearn.decomposition import PCA
from umap import UMAP
available_projection_methods = {
    'PCA': {'module': PCA, 'OOS':False}, # OOS: flag to signal whether or not out of sample is possible
    'UMAP': {'module': UMAP, 'OOS':True},
}
try: 
    from openTSNE import TSNE
    available_projection_methods["TSNE"] = {'module': TSNE, 'OOS':False}
except ImportError:
    print("To support TSNE dataset, please install 'openTSNE': 'pip install openTSNE'.")


class ScatterPlotWidget(widgets.VBox):
    
    embedding = traitlets.Any().tag(sync=True)
    cluster = traitlets.Any().tag(sync=True)
    hover_idcs = traitlets.Any().tag(sync=True)
    # modality1_label = traitlets.Unicode().tag(sync=True)
    # modality2_label = traitlets.Unicode().tag(sync=True)

    mark_colors = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33", "#a65628", "#f781bf", "#999999"]

    def __init__(self, seed=31415, modality1_label='Image', modality2_label='Text', hover_callback=None, unhover_callback=None):
        super(ScatterPlotWidget, self).__init__()

        self.hover_callback = hover_callback
        self.unhover_callback = unhover_callback
        self.seed=seed

        self.nr_components_widget = widgets.BoundedIntText(
            value=2,
            min=2,
            max=10,
            step=1,
            description='Nr Components:',
            disabled=False,
            layout=widgets.Layout(width="150px")
        )
        self.nr_components_widget.observe(self.onUpdateValue, 'value')

        self.x_component_widget = widgets.BoundedIntText(
            value=1,
            min=1,
            max=10,
            step=1,
            description='X component:',
            disabled=False,
            layout=widgets.Layout(width="150px")
        )
        self.x_component_widget.observe(self.update_scatter, 'value')

        self.y_component_widget = widgets.BoundedIntText(
            value=2,
            min=1,
            max=10,
            step=1,
            description='Y component:',
            disabled=False,
            layout=widgets.Layout(width="150px")
        )
        self.y_component_widget.observe(self.update_scatter, 'value')

        traitlets.dlink((self.nr_components_widget, 'value'), (self.x_component_widget, 'max'))
        traitlets.dlink((self.nr_components_widget, 'value'), (self.y_component_widget, 'max'))


        self.fig_widget = go.FigureWidget()
        # self.fig_widget.add_trace(go.Scatter(name = modality1_label, x=[0,1,2,3], y=[0,1,2,3], mode="markers", marker_color=self.mark_colors[0]))
        # self.fig_widget.add_trace(go.Scatter(name = modality2_label, x=[3,2,1,0], y=[0,1,2,3], mode="markers", marker_color=self.mark_colors[1]))
        self.fig_widget.update_layout(width=400, 
                                      height=300, 
                                      margin=dict(l=10, r=10, t=10, b=10),
                                      legend=dict(
                                            yanchor="top",
                                            y=0.99,
                                            xanchor="left",
                                            x=0.01  
                                            )
                                      )

        # self.scatter_image = self.fig_widget.data[0]
        # self.scatter_text = self.fig_widget.data[1]
        
        # self.modality1_label = modality1_label
        # self.modality2_label = modality2_label

        self.select_projection_method = widgets.Dropdown(
            description='Method: ',
            value='UMAP',
            options=list(available_projection_methods),
        )
        self.select_projection_method.observe(self._update_projection_method, 'value')

        self.use_oos_projection = widgets.Checkbox(
            value=False,
            description='Use out of sample projection',
            disabled=not available_projection_methods[self.select_projection_method.value]['OOS'],
            indent=False
        )
        self.use_oos_projection.observe(self._update_projection_method, 'value')

        
        self.children = [self.select_projection_method, self.use_oos_projection, self.nr_components_widget, widgets.HBox([self.x_component_widget, self.y_component_widget]), self.fig_widget]
        

    def _update_projection_method(self, change):
        print('', available_projection_methods[self.select_projection_method.value]['OOS'])
        self.use_oos_projection.disabled = not available_projection_methods[self.select_projection_method.value]['OOS']
        self.onUpdateValue(change)

    # @traitlets.validate("modality1_label", "modality2_label")
    # def _validate_modality_label(self, proposal):
    #     # TODO: validate modality label
    #     return proposal.value
    
    # @traitlets.observe("modality1_label", "modality2_label")
    # def onUpdateModalityLabel(self, change):
    #     self.scatter_image.name = self.modality1_label
    #     self.scatter_text.name = self.modality2_label
        # self.fig_widget.update_layout(legend=dict(
        #                                     yanchor="top",
        #                                     y=0.99,
        #                                     xanchor="left",
        #                                     x=0.01  
        #                                     )
        #                               )

    @traitlets.validate("embedding")
    def _validate_value(self, proposal):
        # print("TODO: validate embedding")
        # for backwards compatibility map array to dict
        if isinstance(proposal.value, np.ndarray):
            embeddings = {}
            embeddings["Modality1"] = proposal.value[:int(len(proposal.value)/2),:]
            embeddings["Modality2"] = proposal.value[int(len(proposal.value)/2):,:]
            return embeddings
        return proposal.value

    @traitlets.observe("embedding")
    def onUpdateValue(self, change):
        projection_method = available_projection_methods[self.select_projection_method.value]
        # TODO: add UI for distance metric
        if self.select_projection_method.value == 'PCA':
            projection = projection_method['module'](n_components=self.nr_components_widget.value, random_state=self.seed)
        else:
            projection = projection_method['module'](n_components=self.nr_components_widget.value, metric="cosine", random_state=self.seed)
            
        if not self.use_oos_projection.disabled and self.use_oos_projection.value:
            # only possible with umap; fit by one modality only and project other modalities out of sample
            projection.fit(self.embedding[list(self.embedding.keys())[0]])

            # project_by = 'image' # TODO: add user select for this
            # if project_by == "image":
            #     self.image_embedding_projection = projection.fit_transform(self.embedding[:int(len(self.embedding)/2),:])
            #     self.text_embedding_projection = projection.transform(self.embedding[int(len(self.embedding)/2):,:])
            # elif project_by == "text":
            #     self.text_embedding_projection = projection.fit_transform(self.embedding[int(len(self.embedding)/2):,:])
            #     self.image_embedding_projection = projection.transform(self.embedding[:int(len(self.embedding)/2),:])

            # self.embedding_projection = np.concatenate((self.image_embedding_projection, self.text_embedding_projection))
            
        else:
            projection.fit(np.concatenate(list(self.embedding.values())))
            # self.embedding_projection = projection.fit_transform(self.embedding)
            # self.image_embedding_projection = self.embedding_projection[:int(len(self.embedding)/2),:]
            # self.text_embedding_projection = self.embedding_projection[int(len(self.embedding)/2):,:]

        embedding_projection = {}
        for modality in self.embedding.keys():
            embedding_projection[modality] = projection.transform(self.embedding[modality])
        self.embedding_projection = embedding_projection

        self.update_scatter(change)

    @traitlets.observe("hover_idcs")
    def hover_changed(self, change):
        shapes = [line for line in self.fig_widget.layout.shapes if line.name == 'pair_connections']

        stacked_embs = np.stack(list(self.embedding_projection.values()), axis=0)
        # r = (stacked_embs.max((0,1)) - stacked_embs.min((0,1))) * 0.01
        range_x = self.fig_widget.layout.xaxis.range[1] - self.fig_widget.layout.xaxis.range[0]
        range_y = self.fig_widget.layout.yaxis.range[1] - self.fig_widget.layout.yaxis.range[0]
        r = (range_x*0.01, range_y*0.01)
        modalities = list(self.embedding_projection.keys())

        for item in self.hover_idcs:
            if type(item) is int:
                # if a line index is given, we highlight the circles and draw lines to the centroid of all modalities of this line
                line_idx = item
                centroid = stacked_embs[:,line_idx,:].mean(axis=0)

                for i in range(len(modalities)):
                    modality = modalities[i]
                    shapes.append(go.layout.Shape(name='hover_line', 
                                                    type='line', 
                                                    x0=self.embedding_projection[modality][line_idx,self.x_component_widget.value-1], 
                                                    y0=self.embedding_projection[modality][line_idx,self.y_component_widget.value-1], 
                                                    x1=centroid[0], 
                                                    y1=centroid[1], 
                                                    line=dict(color="yellow", width=1)))
                    
                    shapes.append(go.layout.Shape(name="hover_circle",
                                                type="circle",
                                                xref="x",
                                                yref="y",
                                                x0=self.embedding_projection[modality][line_idx,self.x_component_widget.value-1]-r[0],
                                                x1=self.embedding_projection[modality][line_idx,self.x_component_widget.value-1]+r[0],
                                                y0=self.embedding_projection[modality][line_idx,self.y_component_widget.value-1]-r[1],
                                                y1=self.embedding_projection[modality][line_idx,self.y_component_widget.value-1]+r[1],
                                                fillcolor="yellow",
                                                line_color="yellow"))
            elif type(item) is tuple:
                # if a tuple of modality and line index is given, we highlight the circles of the given modality and line index
                modality = item[0]
                line_idx = item[1]
                shapes.append(go.layout.Shape(name="hover_circle",
                                            type="circle",
                                            xref="x",
                                            yref="y",
                                            x0=self.embedding_projection[modality][line_idx,self.x_component_widget.value-1]-r[0],
                                            x1=self.embedding_projection[modality][line_idx,self.x_component_widget.value-1]+r[0],
                                            y0=self.embedding_projection[modality][line_idx,self.y_component_widget.value-1]-r[1],
                                            y1=self.embedding_projection[modality][line_idx,self.y_component_widget.value-1]+r[1],
                                            fillcolor="yellow",
                                            line_color="yellow"))


        self.fig_widget.layout.shapes = shapes

    def _on_hover(self, trace, points, state):
        if len(points.point_inds) < 1:
            return
        line_idx = points.point_inds[0]

        self.hover_idcs = [line_idx]

        if self.hover_callback is not None:
            self.hover_callback(trace, points, state)

    def _on_unhover(self, trace, points, state):
        # self.fig_widget.layout.shapes = [line for line in self.fig_widget.layout.shapes if line.name == 'pair_connections']
        if self.unhover_callback is not None:
            self.unhover_callback(trace, points, state)

    def update_scatter(self, change):
        modalities = list(self.embedding_projection.keys())
        for i, modality in enumerate(modalities):
            x_data = self.embedding_projection[modality][:,self.x_component_widget.value-1]
            y_data = self.embedding_projection[modality][:,self.y_component_widget.value-1]
            if len(self.fig_widget.data) <= i: # add a new trace
                trace = go.Scatter(name=modality, x=x_data, y=y_data, mode="markers", marker_color=self.mark_colors[i], hoverinfo="text")
                self.fig_widget.add_trace(trace)
                self.fig_widget.data[i].on_hover(self._on_hover)
                self.fig_widget.data[i].on_unhover(self._on_unhover)
            else:
                self.fig_widget.data[i].x = x_data
                self.fig_widget.data[i].y = y_data
                self.fig_widget.data[i].name = modality

        # self.scatter_image.x = self.image_embedding_projection[:,self.x_component_widget.value-1]
        # self.scatter_image.y = self.image_embedding_projection[:,self.y_component_widget.value-1]

        # self.scatter_text.x = self.text_embedding_projection[:,self.x_component_widget.value-1]
        # self.scatter_text.y = self.text_embedding_projection[:,self.y_component_widget.value-1]

        lines = []
        for line_idx in range(len(list(self.embedding_projection.values())[0])):
            
            # for i in range(len(modalities)-1):
            #     modality1 = modalities[i]
            #     modality2 = modalities[i+1]
            #     lines.append(go.layout.Shape(name='pair_connections', 
            #                                  type='line', 
            #                                  x0=self.embedding_projection[modality1][line_idx,self.x_component_widget.value-1], 
            #                                  y0=self.embedding_projection[modality1][line_idx,self.y_component_widget.value-1], 
            #                                  x1=self.embedding_projection[modality2][line_idx,self.x_component_widget.value-1], 
            #                                  y1=self.embedding_projection[modality2][line_idx,self.y_component_widget.value-1], 
            #                                  line=dict(color="grey", width=1)))

            centroid = np.stack(list(self.embedding_projection.values()))[:,line_idx,:].mean(axis=0)
            for i in range(len(modalities)):
                modality = modalities[i]
                lines.append(go.layout.Shape(name='pair_connections', 
                                             type='line', 
                                             x0=self.embedding_projection[modality][line_idx,self.x_component_widget.value-1], 
                                             y0=self.embedding_projection[modality][line_idx,self.y_component_widget.value-1], 
                                             x1=centroid[0], 
                                             y1=centroid[1], 
                                             line=dict(color="grey", width=1),
                                             opacity=0.4,))
            
        self.fig_widget.layout.shapes = lines


    @traitlets.validate("cluster")
    def _validate_cluster(self, proposal):
        # takes a list of cluster labels + sizes
        # print("TODO: validate cluster")
        return proposal.value

    @traitlets.observe("cluster")
    def onUpdateCluster(self, change):
        print(change)



class CLIPExplorerWidget(widgets.AppLayout):
    def __init__(self, dataset_name, all_images=None, all_prompts=None, all_data=None, models=None):
        # using "all_images" and "all_prompts" is deprecated; use the "all_data" dict instead; e.g. {"image": all_images, "text": all_prompts}
        ### models... list of strings or instances that inherit from CLIPModelInterface 
        super(CLIPExplorerWidget, self).__init__()
        
        # for backwards compatibility
        if all_data is None:
            all_data = {}
            print('WARNING: using all_images and all_prompts is deprecated; use the "all_data" dict instead; e.g. {"image": all_images, "text": all_prompts}')
            if all_images is not None:
                all_data["image"] = all_images
            if all_prompts is not None:
                all_data["text"] = all_prompts

        if models is None:
            models = am_model.available_CLIP_models

        self.models = {}
        for m in models:
            if type(m) == str:
                self.models[m] = am_model.get_model(m)
            elif issubclass(type(m), am_model.CLIPModelInterface):
                self.models[m.model_name] = m
            else:
                print('skipped', m, 'because it is not string or of type CLIPModelInterface')

        
        self.dataset_name = dataset_name
        self.all_data = all_data
        self.size = len(all_data[list(all_data.keys())[0]])

        # ui select widgets
        self.model_select_widget = widgets.Dropdown(
            description='Model: ',
            value=list(self.models.keys())[0],
            options=list(self.models.keys()),
        )

        m = self.models[self.model_select_widget.value]
        self.available_modalities = set(list(self.all_data.keys())).intersection(m.encoding_functions.keys())
        

        self.close_modality_gap_widget = widgets.Checkbox(
            value=False,
            description='Close modality gap',
            disabled=False,
            indent=False
        )

        self.modality1_select_widget = widgets.Dropdown(
            description='Modality 1: ',
            value=list(self.available_modalities)[0],
            options=list(self.available_modalities),
        )
        
        self.modality2_select_widget = widgets.Dropdown(
            description='Modality 2: ',
            value=list(self.available_modalities)[1],
            options=list(self.available_modalities),
        )
        
        # TODO: make dynamic -> users should be able to add/remove modalities dynamically
        if len(self.available_modalities) != 2:
            self.close_modality_gap_widget.layout.visibility = 'hidden'
        
            modality1_data = list(self.all_data.values())[0]
        else:
            modality1_data = self.all_data[self.modality1_select_widget.value]
            # modality2_data = self.all_data[self.modality2_select_widget.value]

        # output widgets
        self.hover_widget = HoverWidget()

        self.embeddings, self.logit_scale = get_embeddings_per_modality(m, self.dataset_name, self.all_data)
        self.scatter_widget = ScatterPlotWidget(hover_callback=self.scatter_hover_fn, unhover_callback=self.scatter_unhover_fn)#(modality1_label=modality1_data.name, modality2_label=modality2_data.name)
        
        self.heatmap_widget = SimilarityHeatmapClusteringWidget(
            cluster_label_data=modality1_data, 
            hover_callback=self.heatmap_hover_fn)
        
        self.log_widget = widgets.Output()

        # TODO: make dynamic -> users should be able to add/remove modalities dynamically
        if len(self.available_modalities) == 2:
            embedding_modality1 = self.embeddings[self.modality1_select_widget.value]
            embedding_modality2 = self.embeddings[self.modality2_select_widget.value]
            self.scatter_widget.embedding = {self.modality1_select_widget.value: embedding_modality1, self.modality2_select_widget.value: embedding_modality2} 
            # self.scatter_widget.embedding = np.concatenate((embedding_modality1, embedding_modality2))

            # TODO: calculate this for all modality combinations
            modality_distance = get_modality_distance(embedding_modality1, embedding_modality2)
            validation_loss = calculate_val_loss(embedding_modality1, embedding_modality2, self.logit_scale.exp())
            with self.log_widget:
                print('Modality distance: %.2f | Loss: %.2f'%(modality_distance, validation_loss))

            self.heatmap_widget.embedding = {self.modality1_select_widget.value: embedding_modality1, self.modality2_select_widget.value: embedding_modality2} #np.concatenate((embedding_modality1, embedding_modality2))
        else:
            embedding_modality1 = self.embeddings[self.modality1_select_widget.value]
            embedding_modality2 = self.embeddings[self.modality2_select_widget.value]
            self.scatter_widget.embedding = self.embeddings
            self.heatmap_widget.embedding = self.embeddings
            with self.log_widget:
                print("ready")

        # callback functions
        self.model_select_widget.observe(self.model_changed, names="value")
        self.close_modality_gap_widget.observe(self.model_changed, names='value')
        self.modality1_select_widget.observe(self.modality_changed, names="value")
        self.modality2_select_widget.observe(self.modality_changed, names="value")

        # display everyting
        header_list = []
        header_list.append(widgets.HBox([self.model_select_widget, self.close_modality_gap_widget]))
        if len(self.available_modalities) == 2: # TODO: make dynamic -> users should be able to add/remove modalities dynamically
            header_list.append(widgets.HBox([self.modality1_select_widget, self.modality2_select_widget]))
        header_list.append(self.log_widget)
        
        self.header = widgets.VBox(header_list)
        self.header.layout.height = '%ipx'%(40*len(header_list))
        vis_widgets = widgets.HBox([self.heatmap_widget, self.scatter_widget])
        self.center = vis_widgets
        self.right_sidebar = self.hover_widget
        self.height = '700px'


    def model_changed(self, change):

        self.log_widget.clear_output()
        with self.log_widget:
            print("loading...")

        m = self.models[self.model_select_widget.value]
        self.available_modalities = set(list(self.all_data.keys())).intersection(m.encoding_functions.keys())
        self.modality1_select_widget.options = list(self.available_modalities)
        self.modality2_select_widget.options = list(self.available_modalities)

        self.embeddings, self.logit_scale = get_embeddings_per_modality(m, self.dataset_name, self.all_data)
        
        # TODO: make dynamic -> users should be able to add/remove modalities dynamically
        if len(self.available_modalities) == 2:
            self.modality_changed(change)
        else:
            self.scatter_widget.embedding = self.embeddings
            self.heatmap_widget.embedding = self.embeddings
            self.log_widget.clear_output()
            with self.log_widget:
                print('ready')



    def modality_changed(self, change):
        self.log_widget.clear_output()
        with self.log_widget:
            print("loading...")
            
        modality1_data = self.all_data[self.modality1_select_widget.value]
        modality2_data = self.all_data[self.modality2_select_widget.value]

        # self.scatter_widget.modality1_label = modality1_data.name
        # self.scatter_widget.modality2_label = modality2_data.name
        # self.heatmap_widget.modality1_label = modality1_data.name
        # self.heatmap_widget.modality2_label = modality2_data.name
        self.heatmap_widget.cluster_label_data = modality1_data

        embedding_modality1 = self.embeddings[self.modality1_select_widget.value]
        embedding_modality2 = self.embeddings[self.modality2_select_widget.value]

        if self.close_modality_gap_widget.value:
            embedding_modality1, embedding_modality2 = get_closed_modality_gap(embedding_modality1, embedding_modality2)
            # image_embedding, text_embedding = get_closed_modality_gap_rotated(image_embedding, text_embedding)

        self.scatter_widget.embedding = np.concatenate((embedding_modality1, embedding_modality2))
        # self.heatmap_widget.embedding = np.concatenate((embedding_modality1, embedding_modality2))
        self.heatmap_widget.embedding = {self.modality1_select_widget.value: embedding_modality1, self.modality2_select_widget.value: embedding_modality2}

        modality_distance = get_modality_distance(embedding_modality1, embedding_modality2)
        # modality_distance = get_modality_distance_rotated(image_embedding, text_embedding)
        
        validation_loss = calculate_val_loss(embedding_modality1, embedding_modality2, self.logit_scale.exp())

        self.log_widget.clear_output()
        with self.log_widget:
            print('Modality distance: %.2f | Loss: %.2f'%(modality_distance, validation_loss))


    def heatmap_hover_fn(self, x_idx, y_idx, x_modality, y_modality):
        self.hover_widget.valueX = self.all_data[x_modality].getVisItem(x_idx)
        self.hover_widget.valueY = self.all_data[y_modality].getVisItem(y_idx)
        # modality1_data = self.all_data[self.modality1_select_widget.value]
        # modality2_data = self.all_data[self.modality2_select_widget.value]
        # # show hover images/texts
        # if x_modality == "modality1":
        #     self.hover_widget.valueX = modality1_data.getVisItem(x_idx)
        # else:
        #     self.hover_widget.valueX = modality2_data.getVisItem(x_idx)
        
        # if y_modality == "modality1":
        #     self.hover_widget.valueY = modality1_data.getVisItem(y_idx)
        # else:
        #     self.hover_widget.valueY = modality2_data.getVisItem(y_idx)

        self.scatter_widget.hover_idcs = [(x_modality, x_idx), (y_modality, y_idx)]

    def scatter_hover_fn(self, trace, points, state):
        if len(points.point_inds) < 1:
            return
        idx = points.point_inds[0]
        # print(trace.name, idx) # image vs text trace
        modality1_data = self.all_data[self.modality1_select_widget.value]
        modality2_data = self.all_data[self.modality2_select_widget.value]

        self.hover_widget.valueY = modality1_data.getVisItem(idx)
        self.hover_widget.valueX = modality2_data.getVisItem(idx)
        
        inverse_idcs = np.argsort(self.heatmap_widget.idcs)
        heatmap_idx = inverse_idcs[idx]
        # self.heatmap_widget.hover_idx = [(heatmap_idx, self.size + heatmap_idx), (self.size + heatmap_idx, heatmap_idx)]
        self.heatmap_widget.hover_idx = [(heatmap_idx + i*self.size, heatmap_idx + i*self.size) for i in range(len(self.available_modalities))]


    def scatter_unhover_fn(self, trace, points, state):
        self.heatmap_widget.hover_idx = []

  

In [10]:
# dataset = Triplet_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\text-audio-image\\", batch_size=100)
# all_images, all_prompts, all_audios = dataset.get_data()
print(len(all_images), len(all_prompts), len(all_audios))

my_widget = CLIPExplorerWidget("test_audio", all_data={"text": all_prompts, "image": all_images, "audio": all_audios}, models=[am_model.ImageBind_Model()])
my_widget.scatter_widget.select_projection_method.value = "PCA"
my_widget

C:\Users\Christina\AppData\Local\Temp\ipykernel_25068\2517419370.py:64: FutureWarning:

The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

C:\Users\Christina\AppData\Local\Temp\ipykernel_25068\2517419370.py:64: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

C:\Users\Christina\AppData\Local\Temp\ipykernel_25068\2517419370.py:66: FutureWarning:

The input object of type 'Tensor' is an array-

100 100 100


In [5]:
print(my_widget.scatter_widget.fig_widget)

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': '#e41a1c'},
              'mode': 'markers',
              'name': 'text',
              'type': 'scatter',
              'uid': 'dcdb79e4-1114-47c5-b52d-a29f7925daa5',
              'x': array([-2.7162833, 13.933243 , 13.233078 , -6.0411777, 14.241256 , -6.0056515,
                          -3.999627 , -5.920571 , 14.163344 , -6.411064 , 14.1514225, -3.1142144,
                          -5.6325464, -3.8758807, -6.106634 , 14.309811 , -6.1900234, -4.069507 ,
                          13.656218 , -3.97272  , -6.085362 , 13.657999 , 14.305956 , 14.467724 ,
                          12.817306 , -2.802722 , -6.2790747, 13.746426 , -6.4750338, -3.1691823,
                          -3.296581 , 14.265967 , 14.279663 , -5.4922967, -5.4029884, 12.765366 ,
                          13.41689  , 14.334169 , -3.9057345, -4.116801 , 12.915027 , -6.0458035,
                          -6.409045 , 13.911397 , -4.0170717,

(-7.912668864381715, 15.90535895074646)

In [21]:
TSNE().fit(np.random.rand(100, 100)).transform(np.random.rand(100, 100))

PartialTSNEEmbedding([[ 8.66231252e-01, -1.80334395e+00],
                      [-6.98845549e-02,  4.86421987e-01],
                      [-1.80532078e+00, -4.17764983e-01],
                      [-1.69716285e+00, -4.86806520e-01],
                      [ 6.34820991e-01,  4.77157939e-01],
                      [-7.02025889e-01,  9.50794310e-01],
                      [ 6.78130001e-01,  8.57959178e-01],
                      [-1.88099454e+00,  8.72435976e-01],
                      [-1.56499882e-02,  1.42629699e-01],
                      [ 2.93348297e-01,  2.69271462e+00],
                      [ 1.79785216e+00, -2.89591829e-01],
                      [ 8.25588826e-01, -3.33093071e-01],
                      [-3.73619880e-01, -8.62589914e-01],
                      [-6.13675611e-01,  1.70157657e+00],
                      [ 1.13072201e+00, -1.31279701e+00],
                      [ 1.03003015e+00,  9.68657049e-01],
                      [-1.19196244e+00, -7.00524880e-01],
              

In [23]:

embeddings, logit_scale = get_embeddings_per_modality(am_model.ImageBind_Model(), "test_audio", {"image": all_images, "text": all_prompts, "audio": all_audios}, batch_size=100)

heatmap = SimilarityHeatmapClusteringWidget(cluster_label_data=all_prompts)
heatmap.embedding = {"image": np.array(embeddings["image"]), "text": np.array(embeddings["text"]), "audio": np.array(embeddings["audio"])}
heatmap


100 100 100
found cached embeddings for test_audio_ImageBind_huge image
found cached embeddings for test_audio_ImageBind_huge text
found cached embeddings for test_audio_ImageBind_huge audio


SimilarityHeatmapClusteringWidget(children=(HBox(children=(Checkbox(value=False, description='Cluster matrix b…

In [23]:
heatmap.value.shape

(200, 200)

In [18]:
similarities_all = get_similarities_all(heatmap.embedding)
len(similarities_all)

300

In [21]:
(similarities_all[np.concatenate([heatmap.idcs + i*heatmap.size for i in range(len(heatmap.modality_labels))], axis=0),:][:,np.concatenate([heatmap.idcs + i*heatmap.size for i in range(len(heatmap.modality_labels))], axis=0)]).shape

(300, 300)

In [3]:

my_widget = am_widgets.CLIPExplorerWidget("test_audio", all_data={"text": all_prompts, "image": all_images, "audio": all_audios }, models=[am_model.ImageBind_Model()]) 
my_widget

C:\Users\Christina\AppData\Local\Temp\ipykernel_30488\3688791554.py:54: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_30488\3688791554.py:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_30488\36

100 100 100


c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


found cached embeddings for test_audio_ImageBind_huge text
found cached embeddings for test_audio_ImageBind_huge image
found cached embeddings for test_audio_ImageBind_huge audio


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

 False


# Image-Thermal

In [ ]:
from glob import glob
from PIL import Image
import numpy as np


class ThermalType(am_data.ImageType):
    name = "Thermal"

    def __init__(self, data) -> None:
        super().__init__(data)


class LLVIP_Dataset(am_data.DatasetInterface):
    name='LLVIP'

    def __init__(self, path, seed=31415, batch_size = 100):
        # download dataset: https://bupt-ai-cz.github.io/LLVIP/
        super().__init__(path, seed, batch_size)
        # path: path to the LLVIP dataset
        image_paths = glob(path + "\\visible\\test\\*.jpg", recursive = True)
        thermal_paths = glob(path + "\\infrared\\test\\*.jpg", recursive = True)
        
        batch_idcs = self._get_random_subsample(len(image_paths))
        image_paths = np.array(image_paths)[batch_idcs]
        thermal_paths = np.array(thermal_paths)[batch_idcs]
        
        all_images = []
        for image_path in image_paths:
            with open(image_path, "rb") as fopen:
                image = Image.open(fopen).convert("RGB")
                all_images.append(image)

            
        all_thermals = []
        
        for thermal_path in thermal_paths:
            with open(thermal_path, "rb") as fopen:
                thermal = Image.open(fopen).convert("L")
                all_thermals.append(thermal)

        self.MODE2_Type = ThermalType

        # TODO... load images and thermals on demand with a custom loader
        self.all_images = np.array(all_images)
        self.all_prompts = np.array(all_thermals)


In [ ]:
# download dataset: https://bupt-ai-cz.github.io/LLVIP/
dataset = LLVIP_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\LLVIP\\", batch_size=100) 
all_images, all_thermals = dataset.get_data()
print(len(all_images), len(all_thermals))

am_widgets.CLIPExplorerWidget("test_thermal", all_data={"image": all_images, "thermal": all_thermals}, models=[am_model.ImageBind_Model()]) 

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:99: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:99: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\32

100 100
found cached embeddings for test_thermal_ImageBind_huge image
batch 1 of 1
torch.Size([100, 1, 224, 224])


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

# Image-Depth

In [ ]:
from glob import glob
from PIL import Image
import numpy as np
import torch
import io
import matplotlib.pyplot as plt


class DepthType(am_data.ImageType):
    name = "Depth"

    def __init__(self, data) -> None:
        # data is an array of (1,224,224) tensors
        super().__init__(data)

    def getVisItem(self, idx):
        output_img = io.BytesIO()
        plt.imsave(output_img, self.data[idx][0], cmap='gray')
        plt.savefig(output_img, format='JPEG')
        return {"displayType": am_data.DisplayTypes.IMAGE, "value": output_img.getvalue()}
    
    

class SUNRGBD_Dataset(am_data.DatasetInterface):
    name='SUNRGBD_NYU'

    def __init__(self, path, seed=31415, batch_size = 100):
        # download "SUNRGBD_V1" dataset from https://rgbd.cs.princeton.edu/
        super().__init__(path, seed, batch_size)
        # path: path to the SUNRGBD dataset
        image_paths = glob(path + "\\kv1\\NYUdata\\NYU*\\fullres\\*.jpg", recursive = True)
        depth_paths = glob(path + "\\kv1\\NYUdata\\NYU*\\fullres\\*.png", recursive = True)
        
        batch_idcs = self._get_random_subsample(len(image_paths))
        image_paths = np.array(image_paths)[batch_idcs]
        depth_paths = np.array(depth_paths)[batch_idcs]
        
        all_images = []
        for image_path in image_paths:
            with open(image_path, "rb") as fopen:
                image = Image.open(fopen).convert("RGB")
                all_images.append(image)

            
        all_depths = []
        for depth_path in depth_paths:
            with open(depth_path, "rb") as fopen:
                depth = Image.open(fopen)
                depth = np.array(depth, dtype=int)
                depth = depth.astype(np.float32) / depth.max() # TODO: need to normalize?
                # depth = depth[np.newaxis,:,:] # need 1 channel -> (1,224,224)
                depth = torch.from_numpy(depth).unsqueeze(0) # need 1 channel -> (1,224,224)
                all_depths.append(depth)

        self.MODE2_Type = DepthType

        # TODO... load images and depths on demand with a custom loader
        self.all_images = np.array(all_images)
        self.all_prompts = np.array(all_depths)



In [ ]:
# download "SUNRGBD_V1" dataset from https://rgbd.cs.princeton.edu/
dataset = SUNRGBD_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\SUNRGBD\\", batch_size=100)
all_images, all_depths = dataset.get_data()
print(len(all_images), len(all_depths))

am_widgets.CLIPExplorerWidget("test_depth", all_data={"image": all_images, "depth": all_depths}, models=[am_model.ImageBind_Model()]) 

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:64: FutureWarning:

The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:64: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:65: FutureWarning:

The input object of type 'Tensor' is an array-

100 100
found cached embeddings for test_depth_ImageBind_huge image
found cached embeddings for test_depth_ImageBind_huge depth


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…